# WINE QUALITY PREDICTION APP USING AWS SAGEMAKER'S IN-BUILT XGBOOST - End-to-End
We will build a Wine Quality Prediction App to help determine the quality of wine from its composition:
- I)   PROBLEM STATEMENT & DATA COLLECTION:

You want to automatically determine the quality of wine from it's underlying components.The data is taken from the UCI datasets and you can get it from the link.  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/

This data will be cleaned and trained using the in-built XGBoost Algorithm on AWS Sagemaker, and an endpoint will be created in AWS ,which wll be used to make predictions when given the inputs on the components in the wine.

- II)  PERFORM EXPLORATORY DATA ANALYSIS 

Inspect the data to validate the quality of the data downloaded from te UCI website. Analyse the distribution of missing values, outliers and gain other relevant insights from the model
- III) DO FEATURE ENGINEERING & SELECTION

Handle the mising values, outliers and do the necessary transformations which will ensure the data is well suited for the machine learning model.And also to maximise the insights gotten from the Exploratory Data Analysis phase.
- IV)  BUILD,TRAIN AND DEPLOY THE MODEL IN SAGEMAKER

The Boto3 package will be used to create the S3 buckets to store the preprocessed dataset.The Sagemaker's inbuilt XGBoost algorithm, will be built, trained and deployed.Including the use of optimal hyperparameters to get the best results for the RMSE( Root Mean Squared Error).An Endpoint will be created after the model is built.
The Endpoint created awill be used to predict the quality of wine when the input compositions are fed to the endpoint.

### V) BUILD,TRAIN AND DEPLOY THE MODEL IN SAGEMAKER
We will perform the following tasks, in order to successully scrape the data we need
- a.) Importing the necessary Libraries and create S3 bucket
- b.) Download the train and test data and store in S3
- c.) Build and Train the Inbuilt XGBoost model
- d.) Deploy the model to an Endpoint
- e.) Test the predictions
- f.) Delete the Endpoint
- g.) Conclusion

#### a.) Importing all the necessary libraries and creating S3 bucket

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.image_uris import retrieve
from sagemaker.session import s3_input, Session

In [2]:
bucket_name = 'winequalityapp' # <--- Give this a unique name, since there can be no 02 bucket names in AWS
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-1


In [3]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [4]:
# set an output path where the trained model will be saved
prefix = 'xgboost-inbuilt-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://winequalityapp/xgboost-inbuilt-algo/output


#### b.) Download the train and test data and store in S3

In [5]:
import pandas as pd

import numpy as np
import urllib

pd.set_option("display.max_columns", None) #setting pandas to display all columns

In [6]:
#Importing the train dataset
try:
    urllib.request.urlretrieve ("https://raw.githubusercontent.com/Bandolo/winequality-studiolab/master/train_clean.csv", "train_clean.csv")
    print('Success: downloaded train_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    train_clean = pd.read_csv('./train_clean.csv')
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded train_clean.csv.
Success: Data loaded into dataframe.


In [11]:
#Importing the test dataset
try:
    urllib.request.urlretrieve ("https://raw.githubusercontent.com/Bandolo/winequality-studiolab/master/test_clean.csv", "test_clean.csv")
    print('Success: downloaded train_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    test_clean = pd.read_csv('./test_clean.csv')
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded train_clean.csv.
Success: Data loaded into dataframe.


In [12]:
print(test_clean.shape)

(320, 12)


In [13]:
print(train_clean.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            9.9             0.540         0.45             2.3      0.071   
1           10.8             0.260         0.45             3.3      0.060   
2            9.9             0.350         0.55             2.1      0.062   
3            5.6             0.850         0.05             1.4      0.045   
4            6.6             0.725         0.09             5.5      0.117   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 16.0                  40.0  0.99910  3.39       0.62   
1                 20.0                  49.0  0.99720  3.13       0.54   
2                  5.0                  14.0  0.99710  3.26       0.79   
3                 12.0                  88.0  0.99240  3.56       0.82   
4                  9.0                  17.0  0.99655  3.35       0.49   

   alcohol  quality2  
0      9.4         1  
1      9.6         1  
2     10.6       

In [14]:
print(test_clean.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0           10.8             0.470         0.43            2.10      0.171   
1            8.1             0.820         0.00            4.10      0.095   
2            9.1             0.290         0.33            2.05      0.063   
3           10.2             0.645         0.36            1.80      0.053   
4           12.2             0.450         0.49            1.40      0.075   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 27.0                  66.0  0.99820  3.17       0.76   
1                  5.0                  14.0  0.99854  3.36       0.53   
2                 13.0                  27.0  0.99516  3.26       0.84   
3                  5.0                  14.0  0.99820  3.17       0.42   
4                  3.0                   6.0  0.99690  3.13       0.63   

   alcohol  quality2  
0     10.8         2  
1      9.6         1  
2     11.7       

In [15]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_clean['quality2'], train_clean.drop(['quality2'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [16]:
# Test Data Into Buckets
pd.concat([test_clean['quality2'], test_clean.drop(['quality2'], 
                                              axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

#### c.) Build and Train the Inbuilt XGBoost model

In [17]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = retrieve('xgboost',boto3.Session().region_name,'latest')

In [21]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.25",
        "gamma":"0.3",
        "min_child_weight":"7",
        "subsample":"1",
        "objective": "multi:softmax",
        "num_class": "10",
        "num_round":50
        }

In [22]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

UnknownServiceError: Unknown service: 'sagemaker-metrics'. Valid service names are: accessanalyzer, account, acm, acm-pca, alexaforbusiness, amp, amplify, amplifybackend, amplifyuibuilder, apigateway, apigatewaymanagementapi, apigatewayv2, appconfig, appconfigdata, appflow, appintegrations, application-autoscaling, application-insights, applicationcostprofiler, appmesh, apprunner, appstream, appsync, athena, auditmanager, autoscaling, autoscaling-plans, backup, backup-gateway, batch, braket, budgets, ce, chime, chime-sdk-identity, chime-sdk-meetings, chime-sdk-messaging, cloud9, cloudcontrol, clouddirectory, cloudformation, cloudfront, cloudhsm, cloudhsmv2, cloudsearch, cloudsearchdomain, cloudtrail, cloudwatch, codeartifact, codebuild, codecommit, codedeploy, codeguru-reviewer, codeguruprofiler, codepipeline, codestar, codestar-connections, codestar-notifications, cognito-identity, cognito-idp, cognito-sync, comprehend, comprehendmedical, compute-optimizer, config, connect, connect-co

In [23]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

NameError: name 'estimator' is not defined

#### d.) Deploy the model to an Endpoint

In [74]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

---------!

#### e.) Test the predictions

In [75]:
#from sagemaker.predictor import csv_serializer
from sagemaker.serializers import CSVSerializer

test_data_array = test_clean.drop(['quality2'], axis=1).values #load the data into an array
#xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(624,)


In [76]:
predictions_array[0:10]

array([ 0.99798775, 10.81545544,  8.42318058, 11.17301559, 10.12899113,
       10.35171318, 10.78121758, 10.18690586, 10.59714508, 10.59128189])

In [77]:
test_clean.head(10)

,Area,Price_log,Awae,Bastos,Bonaberi,Bonamoussadi,Japoma,Kotto,Kribi,Lendi,...,Odza,Olembe,Omnisports,PK12,PK14,PK16,Soa,Village,Yaoundé,Yassa
0,500.0,10.463103,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,500.0,11.002100,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3000.0,7.949091,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,325.0,11.512925,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000.0,9.903488,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,1000.0,10.463103,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,2900.0,11.512925,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1000.0,10.463103,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,482.0,10.463103,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,655.0,11.156251,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
prediction = pd.DataFrame((predictions_array[0:10]),columns=["Predicted Price"])
print(prediction)

   Predicted Price
0              3.0
1          49784.0
2           4551.0
3          71183.0
4          25059.0
5          31311.0
6          48109.0
7          26553.0
8          40020.0
9          39786.0


In [102]:
actual = np.exp(test_clean.Price_log.head(10))
print(actual)

0     35000.0
1     60000.0
2      2833.0
3    100000.0
4     20000.0
5     35000.0
6    100000.0
7     35000.0
8     35000.0
9     70000.0
Name: Price_log, dtype: float64


#### f.) Deleting The Endpoints

In [3]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

NameError: name 'sagemaker' is not defined

Congratulations!!! You just built an end-to-end machine learning app.